# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/root/miniconda3/envs/lyrpeft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/lyrpeft/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/lyrpeft/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.36it/s]
/root/miniconda3/envs/lyrpeft/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、阳气、强盛、刚健等含义。乾卦的卦辞是“元、亨、利、贞”，意味着宇宙初始、通行无阻、充满生机和正义。

乾卦的六爻分别代表了不同的阳爻和阴爻状态，每一卦都有其独特的含义。从下至上，六爻分别为初、二、三、四、五、上，分别对应阳爻和阴爻。通过观察乾卦的六爻组合，我们可以了解到事物的发展变化规律。

乾卦的核心观念是“天行健”，意味着天地运行有其固定的规律，人们应该顺应这一规律，保持事物的发展变化。同时，乾卦还强调“刚健”，即阳刚之气，表示人们应当有坚定的意志和勇气，勇往直前，克服困难，实现目标。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第五卦，也是八卦之一。讼卦由三个阳爻夹两个阴爻构成，象征着诉讼、争端、矛盾等含义。讼卦的卦辞是“元、亨、利、贞”，意味着宇宙初始、通行无阻、充满生机和正义。

讼卦的六爻分别代表了不同的阳爻和阴爻状态，每一卦都有其独特的含义。从下至上，六爻分别为初、二、三、四、五、上，分别对应阳爻和阴爻。通过观察讼卦的六爻组合，我们可以了解到事物的发展变化规律。

讼卦的核心观念是“天听”，意味着天地有耳，能听到人们的呼声。人们应该顺应天意，处理事物时要公平、公正、合法，以达到和谐共处的目标。同时，讼卦还强调“利”，即追求利益、福祉，表示人们应当以合理的方式处理争端，以求达到共赢的结果。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250826_143409"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [11]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [12]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250826_143409）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [13]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250826_143409）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [14]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '占卜师卦是一个古老的中国卜卦方法，旨在预测未来的走向。师卦是由坎卦（水）和坤卦（地）相叠而成，象征着军队的力量和力量对比。这一卦象预示着占问者将面临重要的选择，需要谨慎行事，以地主的身份和地上的情况相结合，才能获得美好的结果。', 'content': '\n师卦的哲学理念是：地载天时，地行，地气和平，地止观养。地象征着大地，孕育万物，有着广泛的象征意义。在这一卦象中，君子观地上的情况，从而行动，获得吉祥。\n\n师卦的吉凶状况各有不同，凶卦主要是观凶，吉卦则是观福。凶卦中，提示占问者需要警惕，谨慎行事，避免决策错误。吉卦中，提示占问者行动谨慎，行事光明磊落，待时机成熟，采取行动。\n\n师卦的运势走向，与个人性格、事业运势、人际关系等方面有关。总体而言，师卦鼓励人们要谨慎行事，等待时机，遵守正道，勇于担当，勇于行动。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250826_143409）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。
